In [28]:
import glob
from datetime import datetime, timedelta, date
from os import path
from igraph import *

In [17]:
folder = "./data"
ext = "log"
ntot = 50

In [18]:
files = [f for f in glob.glob(folder + "**/*." + ext, recursive=False)]

In [50]:
# Load all log files

min_time = 1E10
max_time = 0
users = {}
count = 0
user_index = {}
for fn in files:
    bname = path.basename(fn)
    idx = bname.rfind('-')
    case_id = bname[0:idx]
    events = []
    with open(fn, 'r') as f:
        lst = list(enumerate(f))
        n = len(lst)

        for i, line in lst:
            if i == 0 or i == n - 1:
                continue
            line = line.strip()
            line = line[1:-2]
            parts = line.split(',')
            time = int(parts[0].split(':')[1])

            evstr = parts[1]
            idx = evstr.find(':')
            evtyp = evstr[0:idx]
            if idx < len(evstr):
                evdat = evstr[idx+1:]
            else:
                evdat = None

            # print(time, evtyp, evdat)
            events += [{"time": time, "type": evtyp, "data": evdat}]

            min_time = min(min_time, time)
            max_time = max(max_time, time)            
    
    # Events are stored last to first in log files, reverting the order
    events.reverse()
    users[case_id] = events
    user_index[case_id] = count
    count += 1

min_delta = timedelta(minutes=30)
start_data = datetime.fromtimestamp(min_time)
end_data = datetime.fromtimestamp(max_time) + min_delta
print(start_data)
print(end_data)  

2019-12-10 09:53:21
2019-12-10 17:26:45


In [61]:
def construct_network(td):
    inf_network = []    
    user_outcome = [0] * ntot
    
    ninf = 0
    nknown = 0
    nmiss = 0
    nsurv = 0
    ndead = 0
    npeer = 0
    for key in users:
        events = users[key]
        if len(events) == 0: continue
        pkey = ""
        infect = None
        has_inf_event = False
        for ev in events:
            date = datetime.fromtimestamp(ev["time"])
            if td <= date: continue
            date = date.strftime('%H:%M:%S')
            data = ev["data"]
            if ev["type"] == "OUT":
                if data == "RECOVERED":
                    user_outcome[user_index[key]] = 2
                    nsurv += 1
                elif data == "DEAD":
                    user_outcome[user_index[key]] = 3                
                    ndead += 1
                if not infect:
                    # Infection edge without origin
                    infect = [{"a":"unk", "b":key, "t":date, "s":"-"}]
                ninf += 1            
            elif ev["type"] == "INF":
                user_outcome[user_index[key]] = 1
                if pkey: 
                    # Duplicated parent, skipping
                    continue
                if "PEER" in data:
                    pstr = data[5:-1]
                    if ":" in pstr:
                        pieces = pstr.split(":")
                        pkey = pieces[0]
                        strain = pieces[1]
                    else:
                        pkey = pstr
                        strain = "0"
                    if pkey in users:
                        # print(strain, pkey, "->", key)
                        infect = [{"a":pkey, "b":key, "t":date, "s": strain}]
                        npeer += 1
                        has_inf_event = True
                    else:
                        infect = [{"a":"unk", "b":key, "t":date, "s": strain}]
                        has_inf_event = False                 
                elif "CASE0" in data:
                    strain = data[6:-1]
                    # print(strain, "0", "->", key)
                    infect = [{"a":"zero", "b":key, "t":date, "s": strain}]
                    has_inf_event = True
                elif "SOURCE" in data:
                    infect = [{"a":"src", "b":key, "t":date, "s": strain}]
                    has_inf_event = True

        if has_inf_event:
            nknown += 1
        else:
            nmiss += 1

        if infect:
            inf_network += infect
        
    print("Total number of cases:", ninf)
    print("Total number of deaths:", ndead)
    print("Total number of survivors:", nsurv)
    print("Number of infections with known source:", nknown)
    print("Number of infections from peer:", npeer)
    print("Number of infections with missing source:", nmiss)
    
    return inf_network, user_outcome

td = start_data
while td <= end_data:
    td += min_delta
    print("*******", td)
    network, outcomes = construct_network(td)
#     print(outcomes)
#     print(network)

    g = Graph(directed=True)
    g.add_vertices(ntot)

    g.vs["outcome"] = outcomes
    color_dict = {0: "Green", 1: "Dark Orange", 2: "Deep Sky Blue", 3: "Magenta"}
    g.vs["color"] = [color_dict[out] for out in g.vs["outcome"]]

    for edge in network:
        n0 = edge["a"]
        n1 = edge["b"]
        if n0 in user_index and n1 in user_index:
            print(user_index[n0], "->", user_index[n1])
            g.add_edges([(user_index[n0], user_index[n1])])

#     print(g)
    
    layout = g.layout("fr")
    p = plot(g, layout = layout, vertex_size = 9, edge_arrow_width = 1, edge_arrow_size = 0.5)
    
    p.save(td.strftime('%H-%M') + ".png")
    

******* 2019-12-10 10:23:21
Total number of cases: 0
Total number of deaths: 0
Total number of survivors: 0
Number of infections with known source: 1
Number of infections from peer: 0
Number of infections with missing source: 14
******* 2019-12-10 10:53:21
Total number of cases: 0
Total number of deaths: 0
Total number of survivors: 0
Number of infections with known source: 1
Number of infections from peer: 0
Number of infections with missing source: 14
******* 2019-12-10 11:23:21
Total number of cases: 0
Total number of deaths: 0
Total number of survivors: 0
Number of infections with known source: 2
Number of infections from peer: 0
Number of infections with missing source: 13
******* 2019-12-10 11:53:21
Total number of cases: 0
Total number of deaths: 0
Total number of survivors: 0
Number of infections with known source: 2
Number of infections from peer: 0
Number of infections with missing source: 13
******* 2019-12-10 12:23:21
Total number of cases: 0
Total number of deaths: 0
Total